<style>div.title-slide {width: 100%;display: flex;flex-direction: row;justify-content:space-between;}</style>
<div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat,<br/>Mohamed Naoufal Mahfoudi,<br/>Thierry Turletti,<br/>and Walid Dabbous</span>
</div>

# R2lab's radiomap

## Purpose

R2lab's radiomap is a set of measurements that act as a calibration of the testbed.
The goal is to measure and visualize **received power** at **all node locations** 
when a radio signal is sent **from any given sender** node.

Additionally, that same experiment is carried out with **various settings** 
for the emitted signal, like emission power, Tx rate, channel frequency, 
and with various antenna setups (single antenna, multiple antennas).

## Workflow

End to end experiment involves 2 successive stages:

1. **data acquisition** : per se, including post-processing (aggregation); this can be carried out with the `acquiremap.py` python script, or interactively through the first part of the present `visumap.ipynb` notebook
1. **visualization** : interactively, through the second part of this notebook

For convenience, this git repository also features a directory `datasample` that contains one dataset obtained by running the first-stage acquisition script, so that visualization can be performed right away, as a way to give a quick sense of the results. 

## Notebook Installation 

Here's a list of instructions for setting up the notebook tool jupyter (on OSX):

```
$ sudo pip3 install jupyter 
$ sudo pip3 install --upgrade notebook
$ jupyter nbextension enable --py --sys-prefix widgetsnbextension
$ sudo pip3 install plotly 
```

Then start a notebook server as usual
```
$ jupyter notebook
```

You may need to mention this extra option if you run an early release of jupyter-5.0
```
$ jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
```

Likewise, on macOS there may be a need to add this option with jupyter as of June 2017, (regardless of your default browser)

```
$ jupyter notebook NotebookApp.browser=Safari
```

Once the notebook web page shows up, double click on the file `visumap.ipynb` on your browser, and evaluate the commands using "Shift-Enter" (if needed, see other Internet resources about how to use a notebook).

The notebook contains all information about its own capabilities.

# Data acquisition with `acquiremap.py`

This script is designed to expose to the outside 3 levels of scenarios:

* a `one_run` python function, that runs a complete set of measurements on all nodes, with a specific combination of environment settings (like transmission power, number of antennas, and similar)

* a `all_runs` python function, that calls `one_run` with all possible values for the environment settings

* the script itself, when invoked from the command-line, calls `all_runs` with the environment settings described as command-line options.

Additionally, all these functions can be instructed to perform node initializations (load a fresh image on all nodes, and turn off unused nodes). When this feature is turned on with the 2 multiple-runs versions (either `all_runs` or at script-level), nodes initialization is performed only once before the the first invokation of `one_run`.

In [ ]:
# for convenience, we use this notebook extension
# that will reload any imported python module
# if changed from a text editor
%load_ext autoreload
%autoreload 2

## `one_run`

`one_run` performs data collection on all nodes for one given setting of the environment variables. 

In [ ]:
from acquiremap import one_run

In [ ]:
help(one_run)

##### The `one_run` function implements a scenario in which each node sends a number of ping packets to every other node.

In addition, all nodes run a tcpdump process, and at the end of the run, 
every pcap file (called `fit<N>.pcap` at node `N`) is analyzed locally at node N to 
retrieve the RSSI values received from each other node on its antenna(s), and the result 
is stored in file `result-N.txt` (still on node N).

When all nodes are done, the results are fetched from all nodes and centralized on this laptop in a directory called `t{}-r{}-a{}-ch{}` containing all the files retrieved from all nodes, i.e., `fitN.pcap`, `result-N.txt` for all N nodes.

With:

* `t{}` identifies Tx power for sender nodes in dBm (e.g. 5 to 14):
* `r{}` identifies PHY Tx rate used on all nodes (1 to 54 Mbps)
* `a{}` identifies the antenna mask on all nodes
* `ch{}` denotes the WiFi channel used for transmission

At that point, the post-processing function `processmap.py` is invoked 
to generate intermediate files `rssi-<N>.txt` (one per node),
and eventually one consolidated file `RSSI.txt`, that will be used to plot the radiomap.

#### IMPORTANT NOTE:  Both Atheros 93xx (with ath9k driver) and Intel 5300 (with iwlwifi driver) a/b/g/n NICs are now supported in these scripts. However, both cards do no have the same features and in particular, the iwlwifi driver limits the number of wireless stations in the same IBSS (parameter IWLAGN_STATION_COUNT) to a dozen! So, if you run the script with the 37 nodes, you will observe strange behaviors... Also the Intel 5300 cards are not allowed to use the 5GHz band in Ad Hoc mode. One good point is that it is possible with these cards to decrease the TX power to 0dBm (the lower bound for Atheros cards is 5dBm).

##  `all_runs`

This function simply calls `one_run` with several combinations (a cartesian product) of environment settings.

In [ ]:
from acquiremap import all_runs
help(all_runs)

For example, instead of expecting a paramater `tx_power` that is a simple string, it expects parameter `tx_powers` that is a list of `tx_power` strings to consider. So for example

    all_runs(tx_powers=[5], phy_rates=[1, 54], antenna_masks=[3, 7])
    
would result in 4 runs of `one_run` with the 2 possible values for `phy_rates` multiplied by the 2 possible values for `antenna_masks`

## Shell interface

The command-interface lets you essentially call `all_runs` directly from your shell

In [ ]:
#!./acquiremap.py --help

So by default this simple shell script will run the scenario 
with all values for Tx power, PHY Tx rate and Antenna configurations. 
Book R2lab for at least 2 hours to run it.

## Run your own

The first-level directory name for results is a parameter for you to decide on,
it defaults in the python code to `myradiomap` but you are encouraged 
to provide your own in order to isolate your results.

Tweak the following cell to run your own data collection campaign:

In [ ]:
# this plays the same role as 'datasample' -- see below -- but for your own data
datadir = "myradiomap-intel"
slicename = "inria_radiomap"

In [ ]:
# uncomment this to run the data acquisition yourself
# in the local 'myradiomap' subdir
#all_runs(run_name=datadir, tx_powers=[0,15], phy_rates=[1,54], 
#         antenna_masks=[7], channels=[1], node_ids=[1,2,3,4,5,15,20,31,36,37], wireless_driver='iwlwifi',
#         load_images = True, slicename=slicename)

# Plotting R2lab Radio-Maps

## Data naming scheme

The git repository comes with a pre-populated dataset collected by us, in directory `datasample`.
This contains all the RSSI information to run this visualization.
Of course you can also collect your own RSSI data, tweak `datadir` accordingly in this case.

The `datasample` directory contains a collection of `RSSI.txt` files in the following subdirectories:

* datasample/t14-r1-a1-ch1/RSSI.txt
* datasample/t14-r1-a3-ch1/RSSI.txt
* datasample/t14-r1-a7-ch1/RSSI.txt
* datasample/t14-r54-a1-ch1/RSSI.txt
* datasample/t14-r54-a3-ch1/RSSI.txt
* datasample/t14-r54-a7-ch1/RSSI.txt
* datasample/t5-r1-a1-ch1/RSSI.txt
* datasample/t5-r1-a3-ch1/RSSI.txt
* datasample/t5-r1-a7-ch1/RSSI.txt
* datasample/t5-r54-a1-ch1/RSSI.txt
* datasample/t5-r54-a3-ch1/RSSI.txt
* datasample/t5-r54-a7-ch1/RSSI.txt

Where

* `t5` means an emission power of 5dBm

* `r1` means PHY rate=1Mbps

* `a1` means 1 single valid antenna - and so 2 values in RSSI.txt
* `a3` means 2 valid antennas - and so 3 values in RSSI.txt
* `a7` means 3 valid antennas - and so 4 value in RSSI.txt

* `ch1` means channel 1, i.e. 2412 MHz frequency

This naming scheme is implemented in a helper function in `acquiremap`:

In [ ]:
from acquiremap import naming_scheme

In [ ]:
# here's what the naming scheme looks like
naming_scheme('example-run', tx_power=5, phy_rate=1, antenna_mask=3, channel=40)

***

## Preparation

#### Interactive notebook

In [ ]:
# interactive_output is used to refresh
# a visualization based on user input
from ipywidgets import interactive_output

#### Importing plotly

In [ ]:
import plotly
plotly.__version__

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
# using plotly in offline mode is a requirement 
# in interactive mode - too slow otherwise
import plotly.offline as pyoff

pyoff.init_notebook_mode()

#### Importing a few utilities to retrieve RSSI data from files and convert it to arrays

In [ ]:
# to extract data from a RSSI file
from rssi import read_rssi

In [ ]:
# convert data into x, y and z
from r2labplotly import rssi_to_plotly, rssi_to_plotly3D

#### Importing the dashboard

In [ ]:
# this code deals with gory details of ipywidgets
# its only purpose is to to come up with some 
# reasonably compact layout for our control buttons
from dashboard import dashboard

***

## Generating  2D RSSI Radio-Maps 

In [ ]:
# this plays the same role as 'datasample' -- see below -- but for your own data
datadir = "datasample"

With this in place, we can define a first visualization angle that relies on a 2D representation as offered by `plotly`'s `Heatmap` tool:

In [ ]:
def radiomap2D(datadir, sender, power, rate, antenna_mask, channel, rssi_rank):
    # locate corresponding data file
    filename = str(naming_scheme(datadir, power, rate, antenna_mask, channel) / "RSSI.txt")
    # read that file
    rssi_dict = read_rssi(filename, sender, rssi_rank)
    if not rssi_dict:
        return

    X, Y, Z, T = rssi_to_plotly(rssi_dict)
    heatmap = go.Heatmap( x=X, y=Y, z=Z, text=T, zmin=-100, zmax=0, zauto=False, opacity=1)
    axis = [heatmap]
    layout = go.Layout(
        title="R2lab Radio-Map: Rx power (in dBm) when fit{:02d} is transmitting<br>from {}"
              .format(sender, filename))
    figure = go.Figure(data = axis, layout=layout)
    pyoff.iplot(figure)

In [ ]:
# interactively call radiomap2D
interactive_output(radiomap2D, dashboard(datadir))

*****

***

## Generating 3D RSSI Radio-Maps

Likewise, but this time using `plotly`'s `Surface` object:

In [ ]:
def radiomap3D(datadir, sender, power, rate, antenna_mask, channel, rssi_rank):
    # locate corresponding data file
    filename = str(naming_scheme(datadir, power, rate, antenna_mask, channel) / "RSSI.txt")
    # read that file
    rssi_dict = read_rssi(filename, sender, rssi_rank)
    if not rssi_dict:
        return

    X, Y, Z, T = rssi_to_plotly3D(rssi_dict)
    surface = go.Surface(x=X, y=Y, z=Z, text=T, zmin=-100, zmax=0,
                        #connectgaps=False,
                      )
    data = go.Data([surface])
    axis = dict(
        showbackground=True, # show axis background                                                   
        backgroundcolor="rgb(204, 204, 204)", # set background color to grey                          
        gridcolor="rgb(255, 255, 255)",       # set grid line color                                   
        zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color                              
    )
    title = "R2lab Radio-Map: Rx power (in dBm)"\
            "when fit{:02d} is transmitting<br>from {}"\
            .format(sender, filename)
    layout = go.Layout(
        autosize=True,
        title = title,
        scene=go.Scene(                                     
            xaxis=go.XAxis(axis), # set x-axis style                                                  
            yaxis=go.YAxis(axis), # set y-axis style                                                  
            zaxis=go.ZAxis(axis, title="RSSI (dBm)  ")  # set z-axis style                                                  
        )
    )
    
    figure = go.Figure(data=data, layout=layout)
    pyoff.iplot(figure)

In [ ]:
# interactively call radiomap3D
interactive_output(radiomap3D, dashboard(datadir))